# Линейная регрессия

In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
def func(x):
    if x['nevermarr'] == 1:
        if pd.notnull(x['heduc']):
            return x
        else:
            x['heduc'] = -1
            return x
    else:
        return x
#foo['z'] = foo.apply(func(row), axis = 1)
#foo

Посмотрим на распределение целевого признака — уровня заработной платы: 

In [4]:
raw = pd.read_csv('botswana.tsv', sep = '\t', header = 0, parse_dates = [2])
raw.head()
raw['heduc'].isnull().sum()
print raw.shape
data = raw
data['nevermarr'] = data['agefm'].apply(lambda x : 0 if pd.notnull(x) else 1)
data.drop('evermarr', axis=1, inplace=True)
print data['agefm'].isnull().sum()
data['agefm'] = data['agefm'].apply(lambda x : x if pd.notnull(x) else 0)
print data['agefm'].isnull().sum()
#data['heduc'].where(data['nevermarr']  == 1).apply(lambda x : x if pd.notnull(x) else -1 )
data = data.apply(lambda x : func(x), axis = 1)

data['idlnchld_noans'] = data['idlnchld'].apply(lambda x : 0 if pd.notnull(x) else 1)
data['heduc_noans'] = data['heduc'].apply(lambda x : 0 if pd.notnull(x) else 1)
data['usemeth_noans'] = data['usemeth'].apply(lambda x : 0 if pd.notnull(x) else 1)

data['idlnchld'] = data['idlnchld'].apply(lambda x : x if pd.notnull(x) else -1)
data['heduc'] = data['heduc'].apply(lambda x : x if pd.notnull(x) else -2)
data['usemeth'] = data['usemeth'].apply(lambda x : x if pd.notnull(x) else -1)

data = data.dropna(axis=0)
print data.shape[0] * data.shape[1]


(4361, 15)
2282
0
78264


In [5]:
string = ""
string = " + ".join(data.columns.values)
print string

ceb + age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans


In [6]:
print data.head(100)
data['heduc'][:10]
print data['heduc'].isnull().sum()

    ceb  age educ    religion  idlnchld  knowmeth  usemeth  agefm  heduc  \
0     0   18   10    catholic       4.0       1.0      1.0    0.0   -1.0   
1     2   43   11  protestant       2.0       1.0      1.0   20.0   14.0   
2     0   49    4      spirit       4.0       1.0      0.0   22.0    1.0   
3     0   24   12       other       2.0       1.0      0.0    0.0   -1.0   
4     3   32   13       other       3.0       1.0      1.0   24.0   12.0   
5     1   30    5      spirit       5.0       1.0      0.0   24.0    7.0   
6     3   42    4       other       3.0       1.0      0.0   15.0   11.0   
7     1   36    7       other       4.0       1.0      1.0   24.0    9.0   
8     4   37   16    catholic       4.0       1.0      1.0   26.0   17.0   
9     1   34    5  protestant       4.0       1.0      1.0   18.0    3.0   
10    3   42   15       other       3.0       1.0      1.0   24.0   16.0   
11    3   29    7      spirit       6.0       1.0      1.0   24.0    7.0   
12    4   45

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4348 entries, 0 to 4360
Data columns (total 18 columns):
ceb               4348 non-null int64
age               4348 non-null int64
educ              4348 non-null object
religion          4348 non-null object
idlnchld          4348 non-null float64
knowmeth          4348 non-null float64
usemeth           4348 non-null float64
agefm             4348 non-null float64
heduc             4348 non-null float64
urban             4348 non-null int64
electric          4348 non-null float64
radio             4348 non-null float64
tv                4348 non-null float64
bicycle           4348 non-null float64
nevermarr         4348 non-null int64
idlnchld_noans    4348 non-null int64
heduc_noans       4348 non-null int64
usemeth_noans     4348 non-null int64
dtypes: float64(9), int64(7), object(2)
memory usage: 611.4+ KB


## Построение модели

In [35]:
m1 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + '\
             'radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m1.fit()
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     208.1
Date:                Mon, 27 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:43:57   Log-Likelihood:                -7713.0
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4309   BIC:                         1.575e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
Intercept                 -1

In [10]:
print  sms.het_breushpagan(fitted.resid, fitted.model.exog)

(1126.2841398970429, 1.4104933848476806e-211, 39.641824170360152, 1.4399529083318361e-247)


In [13]:
m2 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + '\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data)
fitted = m2.fit()
print fitted.summary()
print  sms.het_breushpagan(fitted.resid, fitted.model.exog)
print "F=%f, p=%f, k1=%f" % m1.fit().compare_f_test(m2.fit())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     239.8
Date:                Mon, 27 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:07:27   Log-Likelihood:                -7714.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4314   BIC:                         1.571e+04
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.2267      0.202     -6.

In [15]:
m3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + agefm + heduc + urban + electric + '\
             'bicycle + nevermarr + idlnchld_noans + heduc_noans', 
             data=data)
fitted = m3.fit()
print fitted.summary()
print  sms.het_breushpagan(fitted.resid, fitted.model.exog)
m2.fit().compare_f_test(m3.fit())[1]

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.629
Method:                 Least Squares   F-statistic:                     238.6
Date:                Mon, 27 Jun 2016   Prob (F-statistic):               0.00
Time:                        16:10:07   Log-Likelihood:                -7808.6
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4316   BIC:                         1.589e+04
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         -1.3626      0.206     -6.

1.906265566261508e-41